# Tutorial 7: Complete Workflow Example

This tutorial demonstrates a complete workflow for landscape metrics analysis using GeeAdvance.

## Workflow Overview

1. Authenticate and initialize GEE (Standard Methods)
2. Define region of interest
3. Load land cover data
4. Calculate landscape metrics
5. Download data using geemap (handles large areas with tiling)
6. Visualize results
7. Export metrics and data

In [2]:
# Import required packages
import ee
import geeadvance
import geemap
import pandas as pd
import matplotlib.pyplot as plt

# Authenticate and initialize with project ID
ee.Authenticate()
ee.Initialize(project='spatialgeography')

print("✓ Setup complete!")

✓ Setup complete!


## Step 1: Define Region of Interest

Let's analyze a region in India:

In [ ]:
# Define ROI - Example: Part of Western Ghats
roi = geeadvance.create_bbox(
    min_lon=75.0,
    min_lat=12.0,
    max_lon=76.0,
    max_lat=13.0
)

# Visualize the ROI
Map = geemap.Map()
Map.centerObject(roi, zoom=9)
Map.addLayer(roi, {'color': 'red'}, 'Region of Interest')
Map

## Step 2: Load Land Cover Data

In [ ]:
# Load ESA WorldCover 2020
landcover = geeadvance.load_dataset(
    'ESA/WorldCover/v100',
    start_date='2020-01-01',
    end_date='2020-12-31',
    region=roi
)

# Get land cover classes
lc_classes = geeadvance.get_landcover_classes('ESA/WorldCover/v100')
print("Land cover classes:")
for code, name in lc_classes.items():
    print(f"  {code}: {name}")

## Step 3: Visualize Land Cover

In [ ]:
# Add land cover to map
Map.addLayer(
    landcover.select('Map'),
    {'min': 10, 'max': 100, 'palette': [
        '006400', 'ffbb22', 'ffff4c', 'f096ff',
        'fa0000', 'b4b4b4', 'f0f0f0', '0064c8',
        '0096a0', '00cf75', 'fae6a0'
    ]},
    'Land Cover'
)
Map

## Step 4: Calculate Landscape Metrics

In [ ]:
# Calculate comprehensive metrics
metrics = geeadvance.calculate_metrics(
    landcover.select('Map'),
    roi,
    scale=100  # 100m resolution
)

print("Landscape Metrics:")
print(metrics)

## Step 5: Download Data (Large Area Support)

Using geemap's tiling feature to download large areas without size errors:

In [ ]:
# Download using geeadvance with automatic tiling for large areas
output_file = 'landcover_western_ghats.tif'

geeadvance.download_large_area(
    image=landcover.select('Map'),
    region=roi,
    filename=output_file,
    scale=100,
    crs='EPSG:4326'
)

print(f"✓ Downloaded: {output_file}")

## Step 6: Calculate Area Statistics

In [ ]:
# Calculate area for each land cover class
area_stats = geeadvance.area_metrics(landcover.select('Map'), roi, scale=100)

print("Area Statistics:")
print(area_stats)

## Step 7: Export Results

In [ ]:
# Export metrics to CSV
metrics.to_csv('landscape_metrics.csv', index=False)
print("✓ Metrics exported to landscape_metrics.csv")

# Export to Google Drive (for very large areas)
task_id = geeadvance.export_to_drive(
    landcover.select('Map'),
    'landcover_export',
    region=roi,
    scale=100
)
print(f"✓ Export task started: {task_id}")